In [1]:
'''
!pip install pandas 
!pip install scikit-learn 
!pip install spacy 
!pip install transformers 
!pip install nltk
'''

'\n!pip install pandas \n!pip install scikit-learn \n!pip install spacy \n!pip install transformers \n!pip install nltk\n'

In [2]:
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --- ------------------------------------ 1.0/12.8 MB 6.3 MB/s eta 0:00:02
     ----- ---------------------------------- 1.8/12.8 MB 4.8 MB/s eta 0:00:03
     --------- ------------------------------ 2.9/12.8 MB 4.4 MB/s eta 0:00:03
     ----------- ---------------------------- 3.7/12.8 MB 4.3 MB/s eta 0:00:03
     ------------- -------------------------- 4.5/12.8 MB 4.2 MB/s eta 0:00:02
     ---------------- ----------------------- 5.2/12.8 MB 4.1 MB/s eta 0:00:02
     ------------------ --------------------- 6.0/12.8 MB 4.1 MB/s eta 0:00:02
     --------------------- ------------------ 6.8/12.8 MB 4.1 MB/s eta 0:00:02
     ------------------------ --------------- 7.9/12.8 MB 4.1 MB/s eta 0:00:02
     --------------------------- ------------ 8.7/12.8 MB 4.1 MB/s eta 0:00:02
     ----------------------------- ---------- 9.4/12.8 MB 4.0 MB/s eta 0:00:01
     ------------------------------- -------- 10.2/12.8 MB 

In [3]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
import spacy
import nltk
# Download the vader_lexicon if you haven't already
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\saksh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
import pandas as pd
import spacy
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from nltk.sentiment import SentimentIntensityAnalyzer
import numpy as np
import warnings

# Suppress the UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')

# Download the vader_lexicon
nltk.download('vader_lexicon')

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Example CV dataset (replace with real data)
data = {
    'cv_text': [
        "Software Engineer with experience in Python, machine learning, and data science. Leadership skills and team collaboration.",
        "Experienced marketer with a passion for creativity. Strong leadership and strategic thinking in brand management.",
        "Detail-oriented accountant with expertise in financial analysis, budgeting, and reporting. Strong teamwork and organizational skills.",
        "Creative graphic designer skilled in Adobe Suite, project management, and teamwork.",
        "Product manager with 10 years of experience in project management and cross-functional collaboration."
    ],
    'personality': ['Conscientiousness', 'Extraversion', 'Conscientiousness', 'Openness', 'Extraversion']
}

# Create a DataFrame
df = pd.DataFrame(data)

# Preprocess text (clean and tokenize)
def preprocess_text(text):
    # Tokenization and lemmatization
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Apply preprocessing to CV texts
df['cleaned_text'] = df['cv_text'].apply(preprocess_text)

# Sentiment analysis (a proxy for emotional stability)
sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    return sia.polarity_scores(text)['compound']

df['sentiment'] = df['cv_text'].apply(get_sentiment)

# Extract features (bag of words and sentiment)
vectorizer = CountVectorizer(max_features=100)  # Limit to 100 features for simplicity
X = vectorizer.fit_transform(df['cleaned_text']).toarray()

# Convert sentiment column to a 2D array for concatenation
sentiment = df['sentiment'].values.reshape(-1, 1)

# Concatenate sentiment as a new feature column
X = np.hstack([X, sentiment])

# Label encode personality
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['personality'])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model (RandomForest in this case)
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Decode the labels back to their original form
y_test_decoded = label_encoder.inverse_transform(y_test)
y_pred_decoded = label_encoder.inverse_transform(y_pred)

# Evaluate the model and print classification report with zero_division handling
print("Classification Report:")
print(classification_report(y_test_decoded, y_pred_decoded, zero_division=1))


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\saksh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Classification Report:
                   precision    recall  f1-score   support

Conscientiousness       0.00      1.00      0.00       0.0
     Extraversion       1.00      0.00      0.00       1.0

         accuracy                           0.00       1.0
        macro avg       0.50      0.50      0.00       1.0
     weighted avg       1.00      0.00      0.00       1.0

